In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


init workflow


In [2]:
BoSSSshell.WorkflowMgm.Init("XESF_WedgeFlow_OptProbStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


Project name is set to 'XESF_WedgeFlow_OptProbStudy'.
Default Execution queue is chosen for the database.
Opening existing database '\\dc3\userspace\sebastian\cluster\XESF_WedgeFlow_OptProbStudy'.


{ Session Count = 6; Grid Count = 6; Path = \\dc3\userspace\sebastian\cluster\XESF_WedgeFlow_OptProbStudy }

In [3]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


2024-03-29 09:28:58Z

In [4]:
var sessions = database.Sessions;
sessions

#0: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0*	3/29/2024 9:30:54 AM	4f077b48...
#1: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb2-iFlx0-wFLx0-bFlx0-Fphi0*	3/29/2024 9:30:36 AM	9b3a17a2...
#2: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb1-iFlx1-wFLx0-bFlx0-Fphi0*	3/29/2024 9:30:10 AM	dffd8ea7...
#3: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb1-iFlx0-wFLx0-bFlx0-Fphi0*	3/29/2024 9:29:44 AM	0c31b409...
#4: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0*	3/29/2024 9:29:20 AM	6be2e383...
#5: XESFWedgeFlow_TwoLs	XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0*	3/29/2024 9:28:58 AM	6799fbd6...


In [5]:
MatrixAssembler aaa = new MatrixAssembler(3);
var si = database.Sessions.Pick(0);
var ti = si.Timesteps;
si.ID

4f077b48-9ac0-4b32-aa3c-2b7e932291a6

## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(c_P -1)(\rho E - k),~k=c_P \text{Ma}^2\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$


For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the BowShock presented, the v-velocity is zero and we have
$$ h_{in} = \frac{\gamma}{\gamma-1} + \frac{1}{2}u_{in}^2 $$
We prescribe the inflow conditions in terms of pressure $p_{in}=1$, density $\rho_{in}=1$ and the Mach number $M_{in}=4$. From those the velocity is computed as 
$$ u_{in} = M_{in} \sqrt{\gamma \frac{p_{in}}{\rho_{in}}}= 4\sqrt{\gamma}$$

In [6]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 2.0;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

6.300000000000001

In [7]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

## Choose which runs to plot

In [9]:
//var sessions = database.Sessions.Where(s => s.Timesteps.Count >0); //plot all 
var ses=sessions.Where(s => (s.SuccessfulTermination )); // the one plotted in the publication
ses


# A Helper Function to obtain different plot formats

In [10]:
using BoSSS.Solution.Tecplot;
using System.IO;
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

## Actual Etnhalpy Error Plot


In [11]:
int count=0;
foreach(var si in ses){
    var enthalpyerrors = new List<double>();
    foreach(var timestep in si.Timesteps){
        var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
        enthalpyerrors.Add(error);
    }
    allEE.Add(enthalpyerrors);
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Pick(enthalpyerrors.Count -1)).GetInternal();
    plot.AddDataGroup(si.Name,lts.TimeStepNumbers,enthalpyerrors, GetFormat(count));
    count++;
    enthalpyerrors.SaveToTextFile($"EnthalpyErrors_{si.Name}.txt");
}
plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -16 
 
 
 
 
 10 -14 
 
 
 
 
 10 -12 
 
 
 
 
 10 -10 
 
 
 
 
 10 -8 
 
 
 
 
 10 -6 
 
 
 
 
 10 -4 
 
 
 
 
 10 -2 
 
 
 
 
 10 0 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 35 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb1-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 XDGWS-15x10-agg0.4-iPrb1-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb2-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 XDGWS-15x10-agg0.4-iPrb2-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 XDGWS-15x10-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGWS-15x10-agg0.4-iPrb1-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 XDGWS-15x10-agg0.4-iPrb1-iFlx0-wFLx0-bFlx0-Fphi0

## Residual History

In [12]:

using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
int count=0;
foreach(var si in ses){
    var enthalpyerrors = new List<double>();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    lts.ResHistory.SaveToTextFile($"ResHistory_{si.Name}.txt");
    lts.ResHistory.SaveToTextFile($"EnResHistory_{si.Name}.txt");
    plot.AddDataGroup("||r|| - "+ si.Name,lts.TimeStepNumbers,lts.ResHistory, GetFormat(count));
    count++;
    plot.AddDataGroup("EnRes_"+si.Name,lts.TimeStepNumbers,lts.EnResHistory, GetFormat(count));
    count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:800,yRes:500)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -7 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 35 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb1-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb1-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGWS-15x10-agg0.4-iPrb1-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 EnRes X DGWS-15x10-agg0.4-iPrb1-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb2-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb2-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGWS-15x10-agg0.4-iPrb2-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 EnRes X DGWS-15x10-agg0.4-iPrb2-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGWS-15x10-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 EnRes X DGWS-15x10-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 EnRes X DGWS-15x10-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 EnRes X DGWS-15x10-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb1-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGWS-15x10-agg0.4-iPrb1-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGWS-15x10-agg0.4-iPrb1-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 EnRes X DGWS-15x10-agg0.4-iPrb1-iFlx0-wFLx0-bFlx0-Fphi0

### Plot ShadowFields

In [13]:
BoSSS.Solution.Application.DeleteOldPlotFiles();
foreach(var si in ses){
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    var flds=lts.GetShadowFields();
    var tp= new Tecplot(flds.Last().GridDat,2);
    tp.PlotFields(si.Name,0,flds);
}